In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import explode,col, split, concat, udf, struct

import re
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import StringType


#start session 
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.81:7077") \
        .appName("nucleotide_div_stream_alva")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",1)\
        .getOrCreate()
        
#spark_context = spark_session.sparkContext
BATCH_INTERVAL_SECONDS = 120

spark_streaming_context = StreamingContext(spark_session.sparkContext, BATCH_INTERVAL_SECONDS)

lines = spark_streaming_context.textFileStream("hdfs://192.168.2.81:9000//user/LDSA/*.vcf.gz")

vcf = lines.filter(lambda line : not  line.startswith('t=VCF'))\
           .filter(lambda line : not line.startswith('##'))\
           .map(lambda line : line.split())

vcf.pprint()

spark_streaming_context.start()

spark_streaming_context.awaitTermination()


TypeError: textFileStream() got an unexpected keyword argument 'process_all'

-------------------------------------------
Time: 2020-06-05 08:02:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:04:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:06:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:08:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:10:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:12:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:14:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:16:00
-------------------------------------------

-------------------------------------------
Time: 2020-06-05 08:18:00
----------

In [ ]:
spark_streaming_context.stop()

Py4JJavaError: An error occurred while calling z:org.apache.spark.streaming.api.python.PythonDStream.callForeachRDD.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after stopping a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:228)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.dstream.ForEachDStream.<init>(ForEachDStream.scala:39)
	at org.apache.spark.streaming.dstream.DStream.org$apache$spark$streaming$dstream$DStream$$foreachRDD(DStream.scala:654)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$foreachRDD$2.apply$mcV$sp(DStream.scala:639)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$foreachRDD$2.apply(DStream.scala:639)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$foreachRDD$2.apply(DStream.scala:639)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.s

In [ ]:
#Get names of sample columns (samples start with HG and NA)
columns = vcf.schema.names
columns = [x for x in columns if re.match("HG*|NA*", x)]

#extract 100 samples for test 
vcf = vcf.drop(*columns[100:len(columns)])

#Make POS and QUAL integer, filter on QUAL 
vcf = vcf.withColumn("POS", vcf["POS"].cast(IntegerType()))\
        .withColumn('QUAL', vcf['QUAL'].cast(IntegerType()))\
        .filter(vcf['QUAL'] > 20)        

#Get names of sample columns 
columns = vcf.schema.names
columns = [x for x in columns if re.match("HG*|NA*", x)]

for sample in columns: 
    vcf = vcf.withColumn(sample, split(col(sample), ":").cast(ArrayType(StringType())).alias(sample)[0])

#Concatenate the genotype columns to one column 
vcf = vcf.withColumn('GENOTYPES',concat(*columns))

#drop the sample columns 
vcf = vcf.drop(*columns) 

#count number of time each allel occurs 
def count_allel(GENOTYPES, ALT):
    #get number of allels 
    n = re.findall('[ATGC.]+', ALT)
    res = []
    for i in range(0,len(n)+1):
        res.append(GENOTYPES.count(str(i)))
    return res

count_allel_udf = udf(count_allel, StringType())

vcf = vcf.withColumn('ALLEL_FREQ', count_allel_udf('GENOTYPES', 'ALT'))

#Calculate Nd
def Nd(ALLEL_FREQ):
    sqrd = [x**2 for x in ALLEL_FREQ]
    diff = [x-y for x, y in zip(sqrd, ALLEL_FREQ)]
    return 1-(sum(diff)/((sum(ALLEL_FREQ)**2)-sum(ALLEL_FREQ)))
    
Nd_udf = udf(Nd, StringType())

vcf = vcf.withColumn('Nd', Nd_udf('ALLEL_FREQ'))

#define the splits (use min max values from POS argument...) 
splits = list(range(0, 57227415, 1000))
bucketizer = Bucketizer(splits=splits, inputCol="POS", outputCol="POS_BUCKET")

#Transform original data into bucket index
vcf = bucketizer.transform(vcf)

vcf = vcf.withColumn('Nd', vcf['Nd'].cast(StringType()))

#Group by 100000 bp interval and calculate per base nucleotide diversity 
vcf_Nd = vcf.groupBy('POS_BUCKET')\
            .agg({'Nd':'avg'})\
            .orderBy('POS_BUCKET')

#Save data to hdfs 
vcf_Nd.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/user/LDSA/output.csv')